In [1]:
import nltk
import random

from nltk.corpus import twitter_samples

In [35]:
fileids = [fileid for fileid in twitter_samples.fileids()]
print(fileids[:2])

['negative_tweets.json', 'positive_tweets.json']


In [4]:
import re

def remove_handle(string):
    regex = r"(^|[^@\w])@(\w{1,15})\b"
    return re.sub(regex, '', string)

In [17]:
def get_wordnet_pos(pos):
    if pos.startswith('J'):
        return wordnet.ADJ
    elif pos.startswith('V'):
        return wordnet.VERB
    elif pos.startswith('N'):
        return wordnet.NOUN
    elif pos.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [31]:
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk.stem import WordNetLemmatizer

def process_string(string):
    string = remove_handle(string) #remove twitter handle
    
    tokenizer = TweetTokenizer()
    word_tokens = tokenizer.tokenize(string)#tokenize the sentence
    
    tagged = nltk.pos_tag(word_tokens)#pos tagging
    
    lemmatizer = WordNetLemmatizer()
    lemmatized = [lemmatizer.lemmatize(w, get_wordnet_pos(pos)) for (w , pos) in tagged]
    
    stop_words = set(stopwords.words('english'))
    filtered_sentence = [w for w in word_tokens if not w in stop_words] #filter stop words
    
    return filtered_sentence

In [32]:
def process_data(data_strings):
    processed_data = []
    
    for string in data_strings:
        processed_data.append(process_string(string))
    
    return processed_data

In [40]:
processed_data = []

for fileid in fileids[:2]:
    strings = twitter_samples.strings(fileid)
    
    p_s = process_data(strings)
    for p in p_s:
        processed_data.append((p, fileid[:3]))

In [41]:
import random
random.shuffle(processed_data)